# iLOrest examples


Version 0.42

## Introduction

This Jupyter Notebook contains iLOrest examples as well as tips and tricks answering recurrent questions from HPE customers.

## Environment preparation

The following cell sets environment variables and checks the connectivity toward the various BMCs used in this notebook.

In [ ]:
########## Environment preparation ((Version: 0.22)) ############


# Set Student ID number
Stud={{ STDID }}
Id=$(id --user --name)

# location and ports variables
let OpenBmcPort=44000+${Stud}
let iLO5SimulatorPort=45000+${Stud}
let ilo5Port=443

CacheDir="${PWD}/iLOrestCache"
LogDir="${PWD}"
LogFile="${LogDir}/iLOrest.log"

#iLO5SimulatorIP=bmcsimulators
iLO5SimulatorIP=ilo5simulators
iLOSimulator=${iLO5SimulatorIP}:${iLO5SimulatorPort}
iLO5SimulatorURI=https://${iLOSimulator}

OpenBmcIP=openbmcsimulators
OpenBmc="${OpenBmcIP}:${OpenBmcPort}"
OpenBmcURI="https://${OpenBmc}"

ilo5IP="ilo5"
ilo5="${ilo5IP}:${ilo5Port}"
ilo5URI="https://${ilo5}"

# Credentials
User="student"
Password='P@ssw0rd!'

# Miscellaneous
alias ilorest='ilorest --nologo --cache-dir ${CacheDir} --logdir ${LogDir}'
alias ResetIlo5Simulator="ssh -t rstsimul@${iLO5SimulatorIP} $Id"
ilorest logout &>/dev/null # Clear cache


# Verify we can reach the remote Bmcs on the right HTTPS ports. 
for bmc in ilo5 OpenBmc iLO5Simulator ; do
    ip="${bmc}IP" ; port=$(echo ${bmc}Port)
    nc -vz  $(eval echo "\$${ip}") $(eval echo "\$${port}") &> /dev/null && 
        echo "$bmc is reachable" \
        || echo "WARNING: Problem reaching $bmc"
done

### iLO 5 login and cache redirection

This [infrastructure](Pictures/ProgrammingRedfishInfraDescription.png) contains only one physical iLO 5 and prevents multiple students to perform SET actions on this unique BMC. Therefore, students have to use a DMTF BMC simulator populated with an iLO 5 mockup.

However, because `iLOrest` populates a cache directory during the login process you need first to authenticate with the iLO 5 and then modify the cache files to have them pointing to the DMTF simulator. This is the role of the next cell.

In [ ]:
ilorest login $ilo5 -u $User -p $Password

# Copy cache in a safe location before logging out to
# minimize opened sessions. iLO 5 is limited to 15 concurrent
# sessions. 
mv ${CacheDir}/cache{,-bck}
ilorest logout

# Move back the cache
mv ${CacheDir}/cache{-bck,}

# Redirect cache toward the iLO 5 Simulator
sed -i "s.${ilo5URI}.${iLO5SimulatorURI}." ${CacheDir}/cache/*

### <a id = "iLO5SimulatorRestart">Restart iLO 5 simulator</a>

If you need or desire to restart you iLO 5 simulator to start this workshop from scratch or for other reasons, run the following cell at any time.

In [ ]:
# iLO 5 Simulator restart
ResetIlo5Simulator

# Verify we can reach the remote Bmcs on the right HTTPS ports. 
for bmc in ilo5 OpenBmc iLO5Simulator ; do
    ip="${bmc}IP" ; port=$(echo ${bmc}Port)
    nc -vz  $(eval echo "\$${ip}") $(eval echo "\$${port}") &> /dev/null && 
        echo "$bmc is reachable" \
        || echo "WARNING: Problem reaching $bmc"
done

## <a id = "BackToList">List of examples</a>

Note that the list of examples present in this Jupyter Notebook is changing overtime. The classification is not following any strict model; it is a mix of Root service and data type locations.

* AccountService
   - [iLO users: Enforce password complexity and minimum password length](#EnforceiLOUsersPasswordComplexity)

* Chassis
   - [Chassis Serial Number](#ChassisSerialNumber)
   - [Power State](#PowerState)
   - [Indicator LED](#IndicatorLED)
   - [Asset Tag](#AssetTag)

* Thermal
   - [Server thermal information](#ThermalInfo)
   - [Thermal configuration](#ThermalConfiguration)

* Managers (iLO)
    - [Enable/Disable SNMPv1 in iLO](#SNMPv1Enabled)
 
* Resource Directory
    - TBD

* CompusterSystem
     - [PostState and DiscoveryComplete](#PostState_And_DiscoveryComplete)
     - [Bios/RBSU Login password](#ManageBiosRbsuPassword)
     
* Security
     - [Security Dashboard](#SecurityDashboard) 
     
* Bios
     - [Server Serial Number](#BiosSerialNumber)

## AccountService

This root service models the iLO User Accounts.

<a id = "EnforceiLOUsersPasswordComplexity">iLO users: Enforce password complexity and minimum password length</a>\
API Reference document keyword: `EnforcePasswordComplexity` \
iLO GUI location: `Security - Access Settings`

In [ ]:
# Select DMTF data type 
ilorest select AccountService

# List properties of the HPE Oem extension in json format 
ilorest list Oem/Hpe --json

# Modify two AccountService parameters
ilorest set Oem/Hpe/EnforcePasswordComplexity=True
ilorest set Oem/Hpe/MinPasswordLength=5

# Retrieve the list of modified paramters
ilorest status

# Uncommnent the following comand to commit changes in the iLO 5 simulator.
# Note that you will not get the same answer as if done on a real iLO 5.
#ilorest commit

# Undo changes (clear cache)
ilorest select AccountService --refresh

<-- [Back to the list of examples](#BackToList) ; [Restart iLO 5 Simulator](#iLO5SimulatorRestart) -->

## Chassis

This section contains parameters related to the `Chassis` [data type](https://hewlettpackard.github.io/ilo-rest-api-docs/ilo5/#chassis-v1_6_0-chassis). A Chassis represents a physical or virtual container of compute resources with attrbutes such as FRU information, power supplies, temperature, etc.

Complex chassis subsystems may have their own data type (i.e. [Thermal](#ThermalInfo))

Chassis examples:
  - Rack
  - Blade enclosure
  - HPE Synergy frame 
  - Compute node chassis

### <a id = "ChassisSerialNumber">Chassis Serial Number</a>

The chassis serial number of an HPE compute node mentionned on an external label/tag with code bar/QRcode is identical to its embedded server serial number (Bios parameter). Note that the serial number of a C7000 enclosure chassis or Synergy frame chassis is unique different to every SN of embedded compute nodes. 

The following cell retrieves the serial number of an HPE compute node chassis and shows explicitely that it is a `read-only` property. The server serial number is presented [here](#BiosSerialNumber).

[API Reference document](https://hewlettpackard.github.io/ilo-rest-api-docs/ilo5/#serialnumber294) keyword: `SerialNumber` \
iLO GUI location: `Information - iLO Overview` 



In [ ]:
# Select data type
ilorest select Chassis

# Get SerialNumber from the Chassis data type
ilorest get SerialNumber

# Note that this property is READ-ONLY
ilorest info SerialNumber

### <a id = "PowerState">Power State</a>

The following cell retrieves the power state and information related to this parameter. Note that this parameter is read-only. To power on or off you'll need to TBD

In [ ]:
# Select data type
ilorest select Chassis

# Get property value
ilorest get PowerState

# Retrieve property info
ilorest info PowerState



### <a id = "IndicatorLED">Indicator LED</a>

TBD

In [ ]:
# Select data type
ilorest select Chassis

# Get property value
ilorest get IndicatorLED

# Retrieve property info
ilorest info IndicatorLED



In [ ]:
# Change the indicator LED state
ilorest set IndicatorLED="Off" --commit

ilorest status


ilorest get IndicatorLED

### <a id = "AssetTag">Asset Tag</a>

TBD

In [ ]:
# Select data type
ilorest select Chassis

# Get property value
ilorest get AssetTag

# Retrieve property info
ilorest info AssetTag



In [ ]:
# Change the Asset Tag property
ilorest set AssetTag="NewAsset Tag" --commit

ilorest status


ilorest get AssetTag

# Clear cache
ilorest select Chassis. --refresh

ilorest status

<-- [Back to the list of examples](#BackToList) ; [Restart iLO 5 Simulator](#iLO5SimulatorRestart) -->

## Thermal

The [Thermal](https://hewlettpackard.github.io/ilo-rest-api-docs/ilo5/#thermal-v1_1_0-thermal) resource describes the system thermal metrics. It represents the properties for temperature and cooling. 

Thermal Information comes from multiple data types. Since iLO 5 firmware 2.30, thermal configuration can be performed from Bios and TBD data types. 

> NOTE: performing a modification of the thermal configuration from the 

### <a id = "ThermalInfo">Server Thermal Information</a>

In [ ]:
# Retrieve server thermal properties
ilorest serverinfo --thermal
echo 

# Retrieve data type used by the server info macro command
ilorest select


In [ ]:
# List HPE specific thermal resources
ilorest list Oem/Hpe --json

In [ ]:
# Modify property. This should work !?
ilorest set Oem/Hpe/ThermalConfiguration="MaxCooling"

ilorest status

### <a id = "ThermalConfiguration">ThermalConfiguration</a>

> **NOTE** This parameter is new with iLO 5 firmware 2.30. 

    API Reference document keyword: `TBD: Not yet documented` \
    iLO GUI location: `TBD: Not yet implemented`

In [ ]:
# Select data type
ilorest select HpeThermalExt
 
#ilorest list --json

<-- [Back to the list of examples](#BackToList) ; [Restart iLO 5 Simulator](#iLO5SimulatorRestart) -->

## Managers

Examples in this section are related to the `Manager` data type as well as data Oem data types located below `/redfish/v1/Managers/` end point like `HpeiLOSnmpService`. 

### <a id = "SNMPv1Enabled">Enable/Disable SNMPv1 in iLO</a>

> **NOTE** Changing the `SNMPv1Enabled` setting appeared in iLO 5 FW 2.30.

    API Reference document keyword: `SNMPv1Enabled` \
    iLO GUI location: `TBD`

In [ ]:
# Select data type
ilorest select HpeiLOSnmpService

# Get current value
ilorest get SNMPv1Enabled
echo

# Set new value
ilorest set SNMPv1Enabled=False

ilorest status

<-- [Back to the list of examples](#BackToList) ; [Restart iLO 5 Simulator](#iLO5SimulatorRestart) -->

## HPE iLO Resource Directory

The HPE iLO Resource Directory is an HPE Oem extention containing the exhaustive list of the data types present the in a Redfish pecific data types. This directory can be used by Redfish clients to find faster find a specific resource in the Redfish tree.

In [ ]:
Type="HpeThermalExt"
#Type="SoftwareInventory"
ilorest types | grep Resource
ilorest select HpeiLOResourceDirectory
echo
ilorest list instances --json | jq -r '.[]|.[]|."@odata.type"' \
         | awk -F. '/'${Type}'/ {print $1}' | sort -u

<-- [Back to the list of examples](#BackToList) ; [Restart iLO 5 Simulator](#iLO5SimulatorRestart) -->

## ComputerSystem

This section contains examples related to the `ComputerSystem` data type.

### <a id = "PostState_And_DiscoveryComplete">PostState and DiscoveryComplete</a>

Use these state parameters to evaluate the current state of a system before modifying or viewing its resources. As explained in this [article](https://developer.hpe.com/blog/master-the-redfish-server-states-to-improve-your-monitoring-and-manageme) certain parameters cannot be modified if the computer is not in a specific state.

Moreover, some parameters cannot be viewed when the entire discovery of all subsystems is not complete. As an example, you will not be able to view or modify SmartArray and logical drive configurations if the `SmartArrayDiscovery` is not complete.

In [ ]:
ilorest select ComputerSystem
# Try to include HpeComputerSystemExt in the Mockup 
ilorest get Oem/Hpe/DeviceDiscoveryComplete
echo
ilorest get Oem/Hpe/PostState 
echo 


<-- [Back to the list of examples](#BackToList) ; [Restart iLO 5 Simulator](#iLO5SimulatorRestart) -->

### <a id = "ManageBiosRbsuPassword">Bios/RBSU Login password</a>

In [ ]:
# Setting a password for the first time
ilorest setpassword FirstTimePassword '""'

# Removing the Bios/RBSU password
ilorest setpassword '""' FirstTimePassowrd

# No commit needed. No iLO nor system reset needed. 
ilorest status


<-- [Back to the list of examples](#BackToList) ; [Restart iLO 5 Simulator](#iLO5SimulatorRestart) -->

## Security

### <a id = "SecurityDashboard">Security Dashboard</a>

The following five examples retrieve different security dashboard properties.

#### Overall security dashboard status

Use keyword `Security Dashboard` or `SecurityDashboard` in the [API reference document](https://hewlettpackard.github.io/ilo-rest-api-docs/ilo5/#hpeilosecuritydashboard-v1_0_0-hpeilosecuritydashboard) to find the overall security dashboard data type.

In [ ]:
# Select data type
ilorest select HpeiLOSecurityDashboard.

# Retrieve overall security dashboard status 
ilorest get 


#### All Security parameters and properties in json format

Use keyword `SecurityParam` to find the corresponding data type in the [API Reference Document](https://hewlettpackard.github.io/ilo-rest-api-docs/ilo5/#hpeilosecurityparam-v1_1_0-hpeilosecurityparam)

In [ ]:
# Select data type
ilorest select HpeiLOSecurityParam.

# List security params in JSON format
ilorest list --json

#### List all security params at risk

To retrieve this list you don't need to select the data type as it is already selected.

In [ ]:
ilorest list --filter SecurityStatus="Risk"

#### Get specific security parameter properties

In [ ]:
ilorest get --filter Name="Require Login for iLO RBSU"

#### Get specific property of a specific security parameter

In [ ]:
ilorest get Ignore --filter Name="Minimum Password Length"

<-- [Back to the list of examples](#BackToList) ; [Restart iLO 5 Simulator](#iLO5SimulatorRestart) -->

## Bios

### <a id = "BiosSerialNumber">Server Serial Number stored in BIOS</a>

In a computer node, the Serial Number stored in the `Bios.` data type corresponds to the server mother board serial number. HPE requires to change this serial number in case of mother board replacement to match the chassis serial number. 

In [ ]:
# Select data type
ilorest select Bios. --refresh
echo

# Make sure SN is read-write
ilorest info Attributes/SerialNumber --latestschema
echo

# Get SerialNumber from Bios data type.
ilorest get Attributes/SerialNumber

echo "Set new SN"
ilorest set Attributes/SerialNumber="NEWSN" --commit

# Reload a fresh simulator cache
ilorest select Bios. --refresh
echo 

# Validate the SN change
ilorest get Attributes/SerialNumber


<-- [Back to the list of examples](#BackToList) ; [Restart iLO 5 Simulator](#iLO5SimulatorRestart) -->

In [ ]:
ilorest logout

## TBD
